<a href="https://colab.research.google.com/github/sanjshine99/6CS005-High-Performance-Computing/blob/main/HPCTask1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Password cracking using multithreading

In [ ]:
%%writefile EncryptSHA512.c
#include <string.h>
#include <stdio.h>
#include <stdlib.h>
#include <crypt.h>
#include <ctype.h>

#define SALT "$6$AS$"

int isValidFormat(const char *str);

int main(int argc, char *argv[]){

    if (argc != 2) {
        fprintf(stderr, "Usage: %s <password>\n", argv[0]);
        return 1;
    }

    if (!isValidFormat(argv[1])) {
        fprintf(stderr, "Error: Password must be in the format 'LLNN' (e.g., 'AB12')\n");
        return 1;
    }

    printf("%s\n", crypt(argv[1], SALT));

    return 0;
}

// Function to validate the password format
int isValidFormat(const char *str) {
    // Check length first
    if (strlen(str) != 4) {
        return 0;
    }

    if (!isupper(str[0]) || !isupper(str[1])) {
        return 0;
    }

    if (!isdigit(str[2]) || !isdigit(str[3])) {
        return 0;
    }

    return 1;
}

Writing EncryptSHA512.c


In [ ]:
%%shell

gcc EncryptSHA512.c -lcrypt -o output

./output AA00

$6$AS$wKDMKDtx/s3ILNkNaRNFIM0w81/weD1UZ8daNhbQBXuj8L.7OY4trHnSraeizmFYrMwjlb1uRTPxu20rqhmMn/


In [ ]:
%%writefile CrackAZ99.c

#define _GNU_SOURCE
#include <stdio.h>
#include <stdlib.h>
#include <pthread.h>
#include <string.h>
#include <unistd.h>
#include <crypt.h>

pthread_mutex_t lock; // Mutex for thread-safe updates
volatile int found = 0; // Flag to signal other threads to terminate

struct ThreadArgs {
    int threadIndex;
    int start;
    int end;
    char *encryptedPassword;
};

char *foundPassword = NULL; // Pointer to hold the found password

void *Decrypt(void *tArg);
void substr(char *dest, char *src, int start, int length);

int main(int argc, char *argv[]) {
    if (argc < 2) {
        fprintf(stderr, "Usage: %s <encryptedPassword>\n", argv[0]);
        return 1;
    }

    char *encryptedPassword = argv[1];

    int threadCount = 4; // Number of threads
    pthread_t threads[threadCount];
    struct ThreadArgs args[threadCount];

    int totalCombinations = 26 * 26 * 100; // Total possible combinations
    int workPerThread = totalCombinations / threadCount;
    int remainingWork = totalCombinations % threadCount;

    pthread_mutex_init(&lock, NULL);

    for (int i = 0; i < threadCount; i++) {
        args[i].threadIndex = i; // Initialize threadIndex
        args[i].encryptedPassword = encryptedPassword;
        args[i].start = i * workPerThread + (i < remainingWork ? i : remainingWork);
        args[i].end = (i + 1) * workPerThread + (i < remainingWork ? i : remainingWork - 1);

        if (pthread_create(&threads[i], NULL, Decrypt, &args[i]) != 0) {
            fprintf(stderr, "Error creating thread %d\n", i);
            return 2;
        }
    }

    for (int i = 0; i < threadCount; i++) {
        pthread_join(threads[i], NULL);
    }

    if (foundPassword != NULL) {
        printf("Results:\n");
        printf("Encrypted Password: '%s'\n", encryptedPassword);
        printf("Decrypted Password: '%s'\n", foundPassword);
        free(foundPassword);
    } else {
        printf("Password not found.\n");
    }

    pthread_mutex_destroy(&lock);
    return 0;
}

void *Decrypt(void *tArg) {
    struct ThreadArgs *args = (struct ThreadArgs *)tArg;
    char salt[7], potentialPass[7], *encrypted;
    substr(salt, args->encryptedPassword, 0, 6);

    struct crypt_data data;
    data.initialized = 0;

    int progressInterval = 10000; // Control how often to print progress, based on the number of attempts

    for (int i = args->start; i <= args->end && !found; i++) {
        if (i % progressInterval == 0) {
            int firstCharIndex = i / (26 * 100);
            int secondCharIndex = (i / 100) % 26;
            int number = i % 100;
            printf("Thread %d working. Current attempt: %c%c%02d\n", args->threadIndex, 'A' + firstCharIndex, 'A' + secondCharIndex, number);
        }

        int firstCharIndex = i / (26 * 100);
        int secondCharIndex = (i / 100) % 26;
        int number = i % 100;

        sprintf(potentialPass, "%c%c%02d", 'A' + firstCharIndex, 'A' + secondCharIndex, number);
        encrypted = crypt_r(potentialPass, salt, &data);

        pthread_mutex_lock(&lock);
        if (!found && strcmp(args->encryptedPassword, encrypted) == 0) {
            foundPassword = strdup(potentialPass);
            found = 1;
            pthread_mutex_unlock(&lock);
            break;
        }
        pthread_mutex_unlock(&lock);
    }
    return NULL;
}

void substr(char *dest, char *src, int start, int length) {
    memcpy(dest, src + start, length);
    *(dest + length) = '\0';
}


Writing CrackAZ99.c


In [ ]:
%%shell
gcc -o CrackAZ99 CrackAZ99.c -lcrypt -lpthread
./CrackAZ99 '$6$AS$wKDMKDtx/s3ILNkNaRNFIM0w81/weD1UZ8daNhbQBXuj8L.7OY4trHnSraeizmFYrMwjlb1uRTPxu20rqhmMn/'

Thread 0 working. Current attempt: AA00
Results:
Encrypted Password: '$6$AS$wKDMKDtx/s3ILNkNaRNFIM0w81/weD1UZ8daNhbQBXuj8L.7OY4trHnSraeizmFYrMwjlb1uRTPxu20rqhmMn/'
Decrypted Password: 'AA00'
